In [1]:
import os, torch
from xml.etree import ElementTree as ET
from torch.utils.data import Dataset,DataLoader,random_split
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image
import matplotlib.pyplot as plt
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split



In [2]:
FOLDER_DATASET = "/home/kk/Desktop/usama/datasets/VMMRdb"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 32
NUM_CLASSES = None
EPOCHS = 15

In [3]:
print(DEVICE)

cuda


# Definations

In [4]:

class CustomDataset(Dataset):
    def __init__(self,X,y,all_classes, root_dir, transform=None ) -> None:
        self.X = X
        self.y = y
        self.root_dir = root_dir
        assert len(self.X) == len(self.y)

        self.transform = transform
        self.label_map = {label: idx for idx, label in enumerate(set(all_classes))}
        for i,label in enumerate(self.label_map.keys()):
          self.label_map[label] = i


    def __len__(self):
        return len(self.X)


    def __getitem__(self, idx):
        img_name = self.X[idx]
        label = self.y[idx]
        img_path = os.path.join(self.root_dir, label, img_name)

        label_arr = label.split('_')
        label = "_".join(label_arr[:-1])

        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        else:
            image = transforms.ToTensor()(image)
        label = torch.tensor(self.label_map[label], dtype=torch.long)
        return image, label

In [5]:
def test_accuracy(model, test_loader):
  with torch.no_grad():
      total_correct = 0
      total_samples = 0
      for images, labels in test_loader:
          images, labels = images.to(DEVICE), labels.to(DEVICE)
          outputs = model(images)
          _, predicted = torch.max(outputs, 1)
          total_correct += (predicted == labels).sum().item()
          total_samples += labels.size(0)

      accuracy = total_correct / total_samples
      print(f'Test Accuracy On best Model: {accuracy:.4f}')

In [6]:
import copy
from tqdm import tqdm

def train(epochs, model, criterion, optimizer, train_loader, valid_loader, test_loader):
  best_model = None
  best_acc = 0

  # Training loop
  for epoch in range(epochs):
      model.train()
      total_train_correct = 0
      total_train_samples = 0
      tqdm_train_loader = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{epochs}')
      for images, labels in tqdm_train_loader:
          images, labels = images.to(DEVICE), labels.to(DEVICE)
          optimizer.zero_grad()
          outputs = model(images)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          _, predicted = torch.max(outputs, 1)
          total_train_correct += (predicted == labels).sum().item()
          total_train_samples += labels.size(0)
      train_accuracy = total_train_correct / total_train_samples

      # Validation loop
      model.eval()
      with torch.no_grad():
          total_correct = 0
          total_samples = 0
          for images, labels in valid_loader:
              images, labels = images.to(DEVICE), labels.to(DEVICE)
              outputs = model(images)
              _, predicted = torch.max(outputs, 1)
              total_correct += (predicted == labels).sum().item()
              total_samples += labels.size(0)

          accuracy = total_correct / total_samples
          print(f'Epoch [{epoch+1}/{epochs}], Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {accuracy:.4f}')

          if (accuracy > best_acc):
            print("New Best Model with Accuracy: ", accuracy)
            best_acc = accuracy
            best_model = copy.deepcopy(model)
            torch.save(best_model,"scratch_training_E"+str(epoch)+"pt")

  print("Training finished.")

  # Testing the model
  model.eval()
  with torch.no_grad():
      total_correct = 0
      total_samples = 0
      for images, labels in test_loader:
          images, labels = images.to(DEVICE), labels.to(DEVICE)
          outputs = best_model(images)
          _, predicted = torch.max(outputs, 1)
          total_correct += (predicted == labels).sum().item()
          total_samples += labels.size(0)

      accuracy = total_correct / total_samples
      print(f'Test Accuracy On best Model: {accuracy:.4f}')
  return best_model

# Dataset Preparation

In [7]:
classes_list = os.listdir(FOLDER_DATASET)
dataset_list = []

for _class in classes_list:
  images = os.listdir(os.path.join(FOLDER_DATASET,_class))
  for image in images:
    class_l =  _class.split("_")
    dataset_list.append((image,_class,"_".join(class_l[:-1])))
dataset_list = np.array(dataset_list)
NUM_CLASSES = len(np.unique(dataset_list[:,2]))

print("Dataset Images:",len(dataset_list), "TOTAL CLASSES: ", NUM_CLASSES)

Dataset Images: 285086 TOTAL CLASSES:  1175


In [8]:
X_train, X_test, y_train, y_test = train_test_split(dataset_list[:,0],dataset_list[:,1], test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = CustomDataset(X_train,y_train,dataset_list[:,2],FOLDER_DATASET, transform)
valid_dataset = CustomDataset(X_val, y_val,dataset_list[:,2], FOLDER_DATASET, transform)
test_dataset = CustomDataset(X_test,y_test,dataset_list[:,2], FOLDER_DATASET, transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Main

In [9]:
model = models.densenet121(weights=None, num_classes=NUM_CLASSES).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
best_model = train(EPOCHS,model,criterion,optimizer,train_loader,valid_loader,test_loader)

Epoch 1/15: 100%|██████████| 5346/5346 [1:06:57<00:00,  1.33it/s]


Epoch [1/15], Training Accuracy: 0.0557, Validation Accuracy: 0.1043
New Best Model with Accuracy:  0.10431976428082852


Epoch 2/15: 100%|██████████| 5346/5346 [1:14:33<00:00,  1.20it/s]


Epoch [2/15], Training Accuracy: 0.2111, Validation Accuracy: 0.3171
New Best Model with Accuracy:  0.3170633319887051


Epoch 3/15: 100%|██████████| 5346/5346 [1:14:54<00:00,  1.19it/s]


...

Epoch 4/15: 100%|██████████| 5346/5346 [1:13:48<00:00,  1.21it/s]


Epoch [4/15], Training Accuracy: 0.5459, Validation Accuracy: 0.5752
New Best Model with Accuracy:  0.5751793324797867


Epoch 5/15: 100%|██████████| 5346/5346 [1:13:49<00:00,  1.21it/s]


Epoch [5/15], Training Accuracy: 0.6355, Validation Accuracy: 0.6212
New Best Model with Accuracy:  0.6212006945297016


Epoch 6/15:  63%|██████▎   | 3360/5346 [45:43<30:14,  1.09it/s]  

In [ ]:
torch.save(best_model,"vmmrdb_pretraining.pt")